# Video Description Generation and Query Retrieval

## Overview

This notebook demonstrates how to generate video descriptions using the **Qwen2.5-VL (Qwen 2.5 Vision-Language model)** via **Ollama** and store their embeddings in **ChromaDB** for efficient semantic search on **Intel® Core™ Ultra Processors**. 

For each video, a description is generated using Ollama's vision model and stored as an embedding in ChromaDB. When a user submits a query, cosine similarity search is performed in ChromaDB to retrieve the most relevant video description. The matching video is then displayed inline.

This sample uses the videos from the [**stepfun-ai/Step-Video-T2V-Eval**](https://huggingface.co/datasets/stepfun-ai/Step-Video-T2V-Eval) Hugging Face dataset.


- Uses Ollama as the GPU backend
- Simpler setup - no complex model loading required
- Uses Qwen2.5-VL vision model through Ollama
- ChromaDB and semantic search functionality

## Workflow

- During the initial data load, videos from the dataset are processed using **Ollama's Qwen2.5-VL vision model**
- The model generates descriptions for each video
- Generated video descriptions are converted into embeddings using **Sentence Transformers** (all-MiniLM-L6-v2 model)
- These embeddings, along with descriptions and video metadata, are stored in a persistent local **ChromaDB** collection
- When a user submits a query, the text is encoded into an embedding and used to perform semantic search (via cosine similarity) over the ChromaDB collection
- The most relevant video description and associated video file are returned and displayed

## Prerequisites

Before running this notebook, ensure you have:
1. **Ollama installed** and running locally
2. **Qwen2.5-VL vision model** pulled in Ollama: `ollama pull llava` or `ollama pull llama3.2-vision` (or any vision-capable model)
3. The video dataset downloaded

## Setup: Install Dependencies

Run this cell first to ensure all required packages are installed in the current environment.

In [ ]:
import sys
import subprocess

# Install dependencies if not already installed
required_packages = [
    "ollama>=0.4.0",
    "chromadb>=1.0.12",
    "sentence-transformers>=4.1.0",
    "opencv-python>=4.8.0",
    "numpy>=1.24.0",
    "tqdm>=4.65.0",
]

print("Checking and installing required packages...")
for package in required_packages:
    try:
        package_name = package.split(">=")[0]
        __import__(package_name.replace("-", "_"))
        print(f"✓ {package_name} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")

print("\n✅ All dependencies are ready!")

## Building Ollama with GPU Support (Vulkan)

For advanced users who want to build Ollama from source with Vulkan GPU acceleration on Windows:

### Installation Steps

1. **Install Vulkan SDK**
   - Download from: https://vulkan.lunarg.com/sdk/home

2. **Install TDM-GCC**
   - Download from: https://github.com/jmeubank/tdm-gcc/releases/tag/v10.3.0-tdm64-2

3. **Install Go SDK**
   - Download Go v1.24.9: https://go.dev/dl/go1.24.9.windows-amd64.msi

4. **Build Ollama**
   ```bash
   # Set environment variables
   set CGO_ENABLED=1
   set CGO_CFLAGS=-IC:\VulkanSDK\1.4.321.1\Include
   
   # Build with CMake
   cmake -B build
   cmake --build build --config Release -j14
   
   # Build Go binary
   go build
   
   # Run Ollama server (Terminal 1)
   go run . serve
   
   # Test with a model (Terminal 2)
   ollama run gemma3:270m
   ```

**Note:** This is for advanced users who want to compile Ollama from source. The pre-built Ollama installation works fine for most users.

## Import necessary packages

In [ ]:
import os
import base64
import random
import shutil
import logging
import chromadb
import warnings
import ollama
from tqdm import tqdm
from IPython.display import Video, display
from sentence_transformers import SentenceTransformer

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")

## Configuration

In [ ]:
# Ollama configuration
OLLAMA_BASE_URL = "http://localhost:11434"
VISION_MODEL = "llama3.2-vision"  # or "llava" or other vision models
EMBEDDING_MODEL = "all-MiniLM-L6-v2"

# Database configuration
DATABASE_PATH = "./Video_descriptions_database_ollama"
COLLECTION_NAME = "Video_descriptions_ollama"

## Get video file paths

In [ ]:
def get_video_paths():
    """
    Select the number of videos to process and return the selected video file paths.

    Returns:
        list: Selected list of video files paths.
    """
    try:
        dataset_folder = "Step-Video-T2V-Eval"
        max_videos_to_select = 128
        video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']
        video_files = []
        
        for root, dirs, files in os.walk(dataset_folder):
            video_files.extend([
                os.path.join(root, f) for f in files 
                if any(f.lower().endswith(ext) for ext in video_extensions)
            ])
        
        total_video_files = len(video_files)
        num_videos_to_select = min(total_video_files, max_videos_to_select)
        
        random.seed(42)
        selected_video_files = random.sample(video_files, num_videos_to_select)
        
        logging.info(f" Total number of video files found: {total_video_files}")
        logging.info(f" Selected {num_videos_to_select} video files")
        
        return selected_video_files
    except Exception as e:
        logging.exception(f" Error while extracting the video paths: {str(e)}")
        return []

In [ ]:
selected_video_files = get_video_paths()

## Initialize models
Initialize the Sentence Transformer model for embeddings.

In [ ]:
def initialize_embedding_model():
    """
    Initialize Sentence Transformer model for generating embeddings.

    Returns:
        SentenceTransformer: The initialized embedding model.
    """
    try:
        logging.info(f" Loading Sentence Transformer Model: {EMBEDDING_MODEL}")
        embedding_model = SentenceTransformer(EMBEDDING_MODEL)
        return embedding_model
    except Exception as e:
        logging.exception(f" Error while loading the embedding model: {str(e)}")
        return None

In [ ]:
embedding_model = initialize_embedding_model()

## Encode video frame to base64
Utility function to encode video frames for Ollama API.

In [ ]:
def encode_video_frame(video_path, frame_time=0):
    """
    Extract and encode a frame from the video as base64.
    
    Args:
        video_path (str): Path to the video file.
        frame_time (float): Time in seconds to extract the frame.
    
    Returns:
        str: Base64 encoded image string or None if extraction fails.
    """
    try:
        import cv2
        
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            logging.error(f" Cannot open video: {video_path}")
            return None
        
        # Set position to specific time
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_number = int(frame_time * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        ret, frame = cap.read()
        cap.release()
        
        if not ret:
            logging.error(f" Cannot read frame from video: {video_path}")
            return None
        
        # Encode frame to base64
        _, buffer = cv2.imencode('.jpg', frame)
        frame_base64 = base64.b64encode(buffer).decode('utf-8')
        
        return frame_base64
    except Exception as e:
        logging.exception(f" Error encoding video frame: {str(e)}")
        return None

## Generate video description using Ollama

In [ ]:
def generate_video_description_ollama(video_path):
    """
    Generate video description using Ollama vision model.
    
    Args:
        video_path (str): Path to the video file.
    
    Returns:
        str: Generated description of the video.
    """
    try:
        # Extract a frame from the middle of the video
        frame_base64 = encode_video_frame(video_path, frame_time=2.0)
        
        if not frame_base64:
            return "Unable to process video"
        
        # Use Ollama API to generate description
        response = ollama.chat(
            model=VISION_MODEL,
            messages=[{
                'role': 'user',
                'content': 'Describe this sports video in detail. Focus on the main activity, people, objects, and setting.',
                'images': [frame_base64]
            }]
        )
        
        description = response['message']['content']
        return description
        
    except Exception as e:
        logging.exception(f" Error generating description with Ollama: {str(e)}")
        return "Error generating description"

## Get or create ChromaDB collection

In [ ]:
def get_or_create_database():
    """
    Connects to or creates a persistent ChromaDB collection.

    Returns:
        tuple: (collection, existing_descriptions)
    """
    try:
        client = chromadb.PersistentClient(path=DATABASE_PATH)
        collection = client.get_or_create_collection(
            name=COLLECTION_NAME,
            metadata={"hnsw:space": "cosine"}
        )
        
        logging.info(" Checking existing descriptions in database...")
        all_items = collection.get(include=["metadatas", "documents"])
        
        existing_descriptions = {}
        for metadata, doc in zip(all_items['metadatas'], all_items['documents']):
            existing_descriptions[metadata['video_filename']] = doc
        
        logging.info(f" Found {len(existing_descriptions)} existing descriptions")
        return collection, existing_descriptions
        
    except Exception as e:
        logging.exception(f" Error while checking database: {str(e)}")
        return None, {}

In [ ]:
collection, existing_descriptions = get_or_create_database()

## Generate and store video descriptions

Each video will be processed:
1. Check if description already exists in database
2. If not, generate description using Ollama
3. Create embedding and store in ChromaDB

In [ ]:
def generate_and_store_video_descriptions(selected_video_files, collection, existing_descriptions, embedding_model):
    """
    Generate and store video descriptions using Ollama.
    
    Args:
        selected_video_files (list): List of video file paths.
        collection: ChromaDB collection object.
        existing_descriptions (dict): Already processed videos.
        embedding_model: Sentence Transformer model.
    """
    try:
        video_descriptions = {}
        
        for video_file in tqdm(selected_video_files, desc="Processing videos"):
            video_filename = os.path.basename(video_file)
            
            # Skip if already processed
            if video_filename in existing_descriptions:
                logging.info(f" Skipping {video_filename} - already in database")
                video_descriptions[video_file] = existing_descriptions[video_filename]
                continue
            
            logging.info(f"\n Processing {video_filename} using Ollama...")
            
            # Generate description using Ollama
            description_text = generate_video_description_ollama(video_file)
            video_descriptions[video_file] = description_text
            
            logging.info(f" Generated description: {description_text}\n")
            
            # Create embedding
            embedding = embedding_model.encode(description_text).tolist()
            
            # Store in ChromaDB
            collection.add(
                embeddings=[embedding],
                documents=[description_text],
                metadatas=[{"video_filename": video_filename}],
                ids=[video_file]
            )
            
            logging.info(f" Added {video_filename} to database\n")
        
        logging.info(f"\n Processed {len(video_descriptions)} videos")
        logging.info(f" Database now has {collection.count()} total descriptions")
        
    except Exception as e:
        logging.exception(f" Error while generating and storing descriptions: {str(e)}")

<div class="alert alert-warning" role="alert">
  Generating video descriptions using Ollama. This may take some time.
</div>

In [ ]:
generate_and_store_video_descriptions(selected_video_files, collection, existing_descriptions, embedding_model)

## Query the database

In [ ]:
def query_videos_descriptions(query, collection, embedding_model):
    """
    Query ChromaDB collection to find similar videos.
    
    Args:
        query (str): User query.
        collection: ChromaDB collection object.
        embedding_model: Sentence Transformer model.
    
    Returns:
        dict: Query results.
    """
    try:
        query_embedding = embedding_model.encode(query).tolist()
        
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=1,
            include=["documents", "metadatas", "distances"]
        )
        
        logging.info(f" Search results for: '{query}'\n")
        
        for doc, metadata, distance in zip(
            results['documents'][0],
            results['metadatas'][0],
            results['distances'][0]
        ):
            similarity_score = 1 - distance
            logging.info(f" Video filename: {metadata['video_filename']}")
            logging.info(f" Similarity score: {similarity_score:.3f}")
            logging.info(f" Distance: {distance:.3f}")
            logging.info(f" Video description: {doc}\n")
        
        return results
        
    except Exception as e:
        logging.exception(f" Error while querying video descriptions: {str(e)}")
        return None

In [ ]:
query = "Give me the video of the birds and blue sea"
results = query_videos_descriptions(query, collection, embedding_model)

## Display the video

In [ ]:
def display_video(results):
    """
    Display the video based on query results.
    
    Args:
        results (dict): Query results.
    """
    try:
        if results and results['ids']:
            video_path = results['ids'][0][0]
            video = Video(video_path, width=600, height=400)
            display(video)
        else:
            logging.info(" No video found")
    except Exception as e:
        logging.exception(f" Error while displaying the video: {str(e)}")

In [ ]:
display_video(results)

## Remove the database

In [ ]:
def delete_database():
    """
    Delete the database directory.
    """
    if os.path.exists(DATABASE_PATH):
        logging.info("Database deletion option available.")
        database_deletion = 'no'  # Change to 'yes' to delete
        
        if database_deletion == 'yes':
            try:
                shutil.rmtree(DATABASE_PATH)
                logging.info(" Database deleted!")
            except Exception as e:
                logging.exception(f" Error while deleting database: {str(e)}")
        else:
            logging.info(" Database not deleted")
    else:
        logging.info(" Database is not available")

In [ ]:
delete_database()

## Dataset Citations

    @misc{ma2025stepvideot2vtechnicalreportpractice,  
      title={Step-Video-T2V Technical Report: The Practice, Challenges, and Future of Video Foundation Model}, 
      author={Guoqing Ma and Haoyang Huang and Kun Yan and Liangyu Chen and Nan Duan and Shengming Yin and Changyi Wan and Ranchen Ming and Xiaoniu Song and Xing Chen and Yu Zhou and Deshan Sun and Deyu Zhou and Jian Zhou and Kaijun Tan and Kang An and Mei Chen and Wei Ji and Qiling Wu and Wen Sun and Xin Han and Yanan Wei and Zheng Ge and Aojie Li and Bin Wang and Bizhu Huang and Bo Wang and Brian Li and Changxing Miao and Chen Xu and Chenfei Wu and Chenguang Yu and Dapeng Shi and Dingyuan Hu and Enle Liu and Gang Yu and Ge Yang and Guanzhe Huang and Gulin Yan and Haiyang Feng and Hao Nie and Haonan Jia and Hanpeng Hu and Hanqi Chen and Haolong Yan and Heng Wang and Hongcheng Guo and Huilin Xiong and Huixin Xiong and Jiahao Gong and Jianchang Wu and Jiaoren Wu and Jie Wu and Jie Yang and Jiashuai Liu and Jiashuo Li and Jingyang Zhang and Junjing Guo and Junzhe Lin and Kaixiang Li and Lei Liu and Lei Xia and Liang Zhao and Liguo Tan and Liwen Huang and Liying Shi and Ming Li and Mingliang Li and Muhua Cheng and Na Wang and Qiaohui Chen and Qinglin He and Qiuyan Liang and Quan Sun and Ran Sun and Rui Wang and Shaoliang Pang and Shiliang Yang and Sitong Liu and Siqi Liu and Shuli Gao and Tiancheng Cao and Tianyu Wang and Weipeng Ming and Wenqing He and Xu Zhao and Xuelin Zhang and Xianfang Zeng and Xiaojia Liu and Xuan Yang and Yaqi Dai and Yanbo Yu and Yang Li and Yineng Deng and Yingming Wang and Yilei Wang and Yuanwei Lu and Yu Chen and Yu Luo and Yuchu Luo and Yuhe Yin and Yuheng Feng and Yuxiang Yang and Zecheng Tang and Zekai Zhang and Zidong Yang and Binxing Jiao and Jiansheng Chen and Jing Li and Shuchang Zhou and Xiangyu Zhang and Xinhao Zhang and Yibo Zhu and Heung-Yeung Shum and Daxin Jiang},
      year={2025},
      eprint={2502.10248},
      archivePrefix={arXiv},
      primaryClass={cs.CV},
      url={https://arxiv.org/abs/2502.10248}, 
    }